# Advanced Usage

For operations that are not exposed by the `DataPortal` class, you must use the service layer directly.
This can be instantiated with the `Cirro` class. 

In [2]:
from cirro.cirro_client import CirroAPI

cirro = CirroAPI()

The services are exposed and separated out by each section. Look at the autocomplete options, or inspect the [cirro_client.py](../cirro/cirro_client.py) file.

### Examples

In [5]:
# Project / dataset used for the examples
test_project = cirro.projects.list()[-1]
dataset_id = '<dataset_id>'

##### Adding a user to a project

In [14]:
from cirro_api_client.v1.models import ProjectRole

cirro.projects.set_user_role(project_id=test_project.id,
                             username='cirro@cirro.bio',
                             role=ProjectRole.COLLABORATOR)

##### Retrieving a list of running tasks for an analysis

In [3]:
tasks = cirro.execution.get_tasks_for_execution(project_id=test_project.id,
                                                dataset_id=dataset_id)

print(f'Task Name: {tasks[0].name}')
print(f'Status: {tasks[0].status}')
print(f'Stopped At: {tasks[0].stopped_at.isoformat()}')

Task Name: extractStats
Status: COMPLETED
Stopped At: 2024-02-16T02:49:08.916000+00:00


##### Retrieving the log output of a task

In [9]:
logs = cirro.execution.get_task_logs(project_id=test_project.id,
                                     dataset_id=dataset_id,
                                     task_id=tasks[0].native_job_id)
# Print first 5 lines of log output
print('\n'.join(logs.splitlines()[0:5]))

Processing lane 1
Processing tile 1101
Processing cycle 0
Processing cycle 1
Processing cycle 2


##### Listing the jobs that are currently running in a project

In [3]:
cirro.execution.get_project_summary(project_id=test_project.id)

{'Cirro-Spot-11a0fa10': [],
 'Cirro-Head-11a0fa10': [],
 'Cirro-OnDemand-11a0fa10': [],
 'Cirro-Dragen-11a0fa10': []}

##### Retrieving the costs for a project

In [11]:
metrics = cirro.metrics.get_for_project(project_id=test_project.id)
latest_cost = metrics.costs[-1]
print(f'Date: {latest_cost.date}')
print(f'Services: {latest_cost.services.additional_properties}')

Date: 2024-02-01
Services: {'Other': 4.96, 'EC2 - Other': 0.01, 'Amazon Elastic Compute Cloud - Compute': 0.06, 'Amazon SageMaker': 0.01, 'Amazon Simple Storage Service': 0.09, 'AmazonCloudWatch': 0.0}


### Lower-level API Client

If you need to call certain endpoints that are not exposed by the SDK, use the lower-level API Client.
You can access the client by using the `api_client` property on the `Cirro` class.


The API client mirrors the [OpenAPI specification](https://api.dev.cirro.bio/openapi/views/redoc/index.html) of Cirro. It exposes every API endpoint under the syntax: `cirro_api_client.v1.api.<tag> import <endpoint>`. Models are exposed at `cirro_api_client.v1.models`.

Each `tag` corresponds to a group of endpoints, such as datasets, metadata, projects, etc., and the `endpoint` corresponds to an operation.

You can read more about the API client [here](https://github.com/CirroBio/Cirro-client-python). 

Each endpoint exposes a synchronous (`sync`) and asynchronous (`async`) function depending on if you use the async features in Python. All functions require you to pass in an instance of the API client (`client` parameter). You can pass in the `cirro.api_client` property here.

In [ ]:
from cirro_api_client.v1.api.datasets import rerun_transform

rerun_transform.sync_detailed(project_id=test_project.id, 
                              dataset_id=dataset_id,
                              client=cirro.api_client)

In [4]:
from cirro_api_client.v1.models import DatasetAssetsManifest
from cirro_api_client.v1.api.datasets import get_dataset_manifest

manifest: DatasetAssetsManifest = get_dataset_manifest.sync(project_id=test_project.id, 
                                                            dataset_id=dataset_id,
                                                            client=cirro.api_client)
manifest.files

[FileEntry(path='data/max_intensity_1.svg', size=67051, metadata=FileEntryMetadata(additional_properties={}), additional_properties={}),
 FileEntry(path='data/percent_base.svg', size=85084, metadata=FileEntryMetadata(additional_properties={}), additional_properties={}),
 FileEntry(path='data/run_metrics.html', size=3809, metadata=FileEntryMetadata(additional_properties={}), additional_properties={}),
 FileEntry(path='data/run_metrics.json', size=2360, metadata=FileEntryMetadata(additional_properties={}), additional_properties={})]